In [13]:
from py2neo import Graph
import pandas as pd

# 1️⃣ --- Kết nối tới Neo4j ---
graph = Graph("neo4j+s://b0276d86.databases.neo4j.io", auth=("neo4j", "_IefyDowcWvSyZrs9v0bIDdJD5rP51rINpoEG-UrLp8"))

In [14]:
# 2️⃣ --- Truy vấn danh sách ứng viên sử dụng ngôn ngữ "English" ---
query = """
MATCH (e:Employee)-[:LANGUAGE]->(l:Language {name: 'English'})
MATCH (e)-[:HAS_LANGUAGE_LEVEL]->(t3:Level {name:'IELTS 6.5, Toeic: 785'})
RETURN e.name AS name
"""
df = pd.DataFrame(graph.run(query).data())
print("Danh sách ứng viên sử dụng ngôn ngữ English:")
print(df)

# Xử lý NaN trong DataFrame
df = df.applymap(lambda x: [] if isinstance(x, float) and pd.isna(x) else x)

Danh sách ứng viên sử dụng ngôn ngữ English:
         name
0  Chu Ngoc M


C:\Users\Chieu\AppData\Local\Temp\ipykernel_17016\3046537366.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: [] if isinstance(x, float) and pd.isna(x) else x)


In [15]:
# 3️⃣ --- Tạo danh sách ứng viên đã lọc ---
filtered_candidates = [{'id': row['name'], 'name': row['name']} for row in df.to_dict('records')]
print("\nDanh sách ứng viên đã lọc:")
print(filtered_candidates)


Danh sách ứng viên đã lọc:
[{'id': 'Chu Ngoc M', 'name': 'Chu Ngoc M'}]


In [16]:
def build_candidate_tree(graph, candidate_name):
    """
    Xây dựng cây phân cấp từ node gốc là ứng viên.
    Chỉ xét các node liên kết trực tiếp với ứng viên.
    Nếu một node đã là con trong nhánh khác thì không xuất hiện lại dưới ứng viên.
    """

    # Bước 1: Lấy danh sách các node liên kết trực tiếp với ứng viên
    query_direct_nodes = f"""
    MATCH (e:Employee {{name: '{candidate_name}'}})-[*1]->(n)
    WHERE NOT n:Employee
    RETURN DISTINCT n.name AS node_name
    """
    result_direct_nodes = graph.run(query_direct_nodes).data()
    direct_nodes = [row['node_name'] for row in result_direct_nodes if row['node_name'] is not None]

    # Bước 2: Truy tất cả các quan hệ cha-con chỉ trong tập direct_nodes
    query_relations = f"""
    MATCH (parent)-[*1]->(child)
    WHERE parent.name IN {direct_nodes} AND child.name IN {direct_nodes}
    RETURN DISTINCT parent.name AS parent_name, child.name AS child_name
    """
    result_relations = graph.run(query_relations).data()

    # Bước 3: Xây dict quan hệ cha → list con
    relations = {}
    for row in result_relations:
        parent = row['parent_name']
        child = row['child_name']
        if parent not in relations:
            relations[parent] = []
        relations[parent].append(child)

    # Bước 4: Hàm đệ quy xây cây con từ 1 node
    def build_subtree(node, visited):
        if node in visited:
            return {}
        visited.add(node)
        children = relations.get(node, [])
        return {child: build_subtree(child, visited) for child in children}

    # Bước 5: Xây tất cả các cây con từ các node trực tiếp
    subtree_map = {}
    for node in direct_nodes:
        subtree_map[node] = build_subtree(node, set())

    # Bước 6: Tìm các node đã là cháu trong cây con → loại khỏi cấp 1
    all_nested_nodes = set()
    def collect_all_nodes(subtree):
        for k, v in subtree.items():
            all_nested_nodes.add(k)
            if isinstance(v, dict):
                collect_all_nodes(v)

    for tree in subtree_map.values():
        collect_all_nodes(tree)

    # Bước 7: Chỉ giữ các node trực tiếp mà không nằm trong cây con khác
    final_subtree = {}
    for node, tree in subtree_map.items():
        if node not in all_nested_nodes:
            final_subtree[node] = tree

    # Bước 8: Gắn cây vào gốc ứng viên
    return {candidate_name: final_subtree}


In [17]:
def print_tree(tree, indent=0):
    """In cây quan hệ với cấu trúc phân cấp."""
    for key, value in tree.items():
        print(" " * indent + f"- {key}")
        if isinstance(value, dict):
            print_tree(value, indent + 4)

In [18]:
# Xây dựng cây quan hệ cho từng ứng viên
for candidate in filtered_candidates:
    candidate['tree'] = build_candidate_tree(graph, candidate['name'])
def print_full_tree(tree, indent=0):
    """In toàn bộ cây quan hệ, không rút gọn."""
    for key, value in tree.items():
        print(" " * indent + f"- {key}")
        if isinstance(value, dict):
            print_full_tree(value, indent + 4)
# In cây quan hệ của từng ứng viên
print("\nCây quan hệ của từng ứng viên:")
for candidate in filtered_candidates:
    print(f"\nỨng viên: {candidate['name']}")
    print_tree(candidate['tree'])


Cây quan hệ của từng ứng viên:

Ứng viên: Chu Ngoc M
- Chu Ngoc M
    - Experienced in working with microservices
    - Experienced in Software Engineering and DevOps workflows
    - 3 years of experience in the Software Engineering
    - Ruby Developer
        - Docker
    - USMH Monitoring
        - PostgresSQL
        - CloudWatch
    - SPPL Microservices platform
    - Azure
    - AWS
    - Linux
    - Windows
    - AWS CloudFormation
    - Grafana
    - Prometheus
    - Bash Script
    - English
        - IELTS 6.5, Toeic: 785
    - Vietnamese
        - Native
    - 4 year
    - Viet Nam


In [19]:
def apply_node_ranking_with_hierarchy(filtered_candidates, critical_nodes):
    """
    Tính điểm cho từng ứng viên dựa trên critical nodes và cây quan hệ.
    """
    all_candidate_names = {candidate['name'] for candidate in filtered_candidates}  # Tập hợp tên tất cả ứng viên

    for candidate in filtered_candidates:
        tree = candidate['tree']
        
        # Tính điểm dựa trên critical nodes và quan hệ cha-con
        total_score = 0
        visited_nodes = set()

        def traverse_tree(node, subtree, depth=1):
            """Duyệt cây để tính điểm."""
            nonlocal total_score
            if node in visited_nodes:
                return

            if node in all_candidate_names:
                # Bỏ qua tính điểm cho ứng viên nhưng vẫn duyệt con
                for child, child_subtree in subtree.items():
                    traverse_tree(child, child_subtree, depth + 1)
                return

            visited_nodes.add(node)

            # Đánh trọng số
            if node in critical_nodes:
                total_score += 10  # Điểm cao nhất cho critical nodes
                print(f"Node '{node}' (critical) được tính 10 điểm.")
            elif any(child in critical_nodes for child in subtree.keys()):  # Node là cha trực tiếp của critical nodes
                total_score += 3
                print(f"Node '{node}' (cha trực tiếp của critical node) được tính 3 điểm.")
            elif depth == 1:  # Node con trực tiếp
                total_score += 5
                print(f"Node '{node}' (con trực tiếp) được tính 5 điểm.")
            else:  # Các node khác
                total_score += 2
                print(f"Node '{node}' (khác) được tính 2 điểm.")

            # Duyệt các node con
            for child, child_subtree in subtree.items():
                traverse_tree(child, child_subtree, depth + 1)

        # Bắt đầu duyệt từ node gốc (ứng viên)
        print(f"\nDuyệt cây cho ứng viên: {candidate['name']}")
        traverse_tree(candidate['name'], tree[candidate['name']])

        candidate['total_score'] = total_score

    # Sắp xếp ứng viên theo điểm số
    filtered_candidates.sort(key=lambda x: x['total_score'], reverse=True)
    return filtered_candidates

In [20]:
# Danh sách critical nodes
critical_nodes = ['IELTS 6.5, Toeic: 785', 'PostgresSQL']  # Thay đổi danh sách này nếu cần

# Áp dụng hàm ranking
ranked_candidates = apply_node_ranking_with_hierarchy(filtered_candidates, critical_nodes)

# Hiển thị kết quả
print("\nKết quả xếp hạng ứng viên:")
for c in ranked_candidates:
    print(f"✅ {c['name']} – Total Score: {c['total_score']}")


Duyệt cây cho ứng viên: Chu Ngoc M
Node 'Experienced in working with microservices' (khác) được tính 2 điểm.
Node 'Experienced in Software Engineering and DevOps workflows' (khác) được tính 2 điểm.
Node '3 years of experience in the Software Engineering' (khác) được tính 2 điểm.
Node 'Ruby Developer' (khác) được tính 2 điểm.
Node 'Docker' (khác) được tính 2 điểm.
Node 'USMH Monitoring' (cha trực tiếp của critical node) được tính 3 điểm.
Node 'PostgresSQL' (critical) được tính 10 điểm.
Node 'CloudWatch' (khác) được tính 2 điểm.
Node 'SPPL Microservices platform' (khác) được tính 2 điểm.
Node 'Azure' (khác) được tính 2 điểm.
Node 'AWS' (khác) được tính 2 điểm.
Node 'Linux' (khác) được tính 2 điểm.
Node 'Windows' (khác) được tính 2 điểm.
Node 'AWS CloudFormation' (khác) được tính 2 điểm.
Node 'Grafana' (khác) được tính 2 điểm.
Node 'Prometheus' (khác) được tính 2 điểm.
Node 'Bash Script' (khác) được tính 2 điểm.
Node 'English' (cha trực tiếp của critical node) được tính 3 điểm.
Node 'IEL

In [21]:
for candidate in filtered_candidates:
    name = candidate['name']
    
    query = f"""
    MATCH (e:Employee {{name: '{name}'}})-[]->(n)
    WHERE NOT n:Employee
    RETURN COUNT(DISTINCT n) AS degree_score
    """
    result = graph.run(query).data()
    degree_score = result[0]['degree_score'] if result else 0

    candidate['pagerank_score'] = degree_score  # không scale *100
    candidate['total_score'] = 0.6 * candidate['total_score'] + 0.4 * degree_score


In [22]:
print("\n🔥 Kết quả xếp hạng ứng viên (tự tính degree-score thay thế PageRank):")
filtered_candidates.sort(key=lambda x: x['total_score'], reverse=True)

for c in filtered_candidates:
    print(f"✅ {c['name']:<20} | Tree Score: {round(c['total_score'] * 10 / 6, 2):>6} | "
          f"Degree Score: {round(c['pagerank_score'], 2):>6} | "
          f"Total Score: {round(c['total_score'], 2):>6}")



🔥 Kết quả xếp hạng ứng viên (tự tính degree-score thay thế PageRank):
✅ Chu Ngoc M           | Tree Score:  80.67 | Degree Score:     25 | Total Score:   48.4


In [23]:
def run_pagerank():
    query = """
    CALL gds.graph.project(
        'myGraph',
        ['Employee', 'Tool', 'Language', 'ProgramingLanguage', 'Cloud'],
        '*'
    );

    CALL gds.pageRank.stream('myGraph')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS name, score
    ORDER BY score DESC
    LIMIT 10
    """
    
    result = graph.run(query)
    for record in result:
        print(record["name"], record["score"])


In [24]:
def describe_candidate(candidate, critical_nodes):
    """
    Tạo mô tả về ứng viên dựa trên cây quan hệ của họ.
    """
    tree = candidate['tree']
    name = candidate['name']
    description = [f"Ứng viên {name} có các ưu điểm nổi bật như sau:"]

    def traverse_tree(node, subtree, parent=None):
        """
        Duyệt qua cây để tạo mô tả.
        """
        if parent:
            # Mô tả quan hệ giữa parent và node
            if node in critical_nodes:
                description.append(f"- {node} (critical node) là một điểm mạnh quan trọng, liên kết trực tiếp với {parent}.")
            else:
                description.append(f"- {node} là một kỹ năng/hệ thống liên kết với {parent}.")
        else:
            # Node gốc (ứng viên)
            description.append(f"- {node} là ứng viên chính.")

        # Duyệt các node con
        for child, child_subtree in subtree.items():
            traverse_tree(child, child_subtree, node)

    # Bắt đầu duyệt từ gốc cây
    traverse_tree(name, tree[name])

    return "\n".join(description)

In [25]:
# Danh sách critical nodes
critical_nodes = ['IELTS 6.5, Toeic: 785', 'PostgresSQL']  # Thay đổi danh sách này nếu cần

# Tạo mô tả cho từng ứng viên
print("\nMô tả ứng viên:")
for candidate in filtered_candidates:
    description = describe_candidate(candidate, critical_nodes)
    print(description)
    print("\n" + "-" * 50 + "\n")


Mô tả ứng viên:
Ứng viên Chu Ngoc M có các ưu điểm nổi bật như sau:
- Chu Ngoc M là ứng viên chính.
- Experienced in working with microservices là một kỹ năng/hệ thống liên kết với Chu Ngoc M.
- Experienced in Software Engineering and DevOps workflows là một kỹ năng/hệ thống liên kết với Chu Ngoc M.
- 3 years of experience in the Software Engineering là một kỹ năng/hệ thống liên kết với Chu Ngoc M.
- Ruby Developer là một kỹ năng/hệ thống liên kết với Chu Ngoc M.
- Docker là một kỹ năng/hệ thống liên kết với Ruby Developer.
- USMH Monitoring là một kỹ năng/hệ thống liên kết với Chu Ngoc M.
- PostgresSQL (critical node) là một điểm mạnh quan trọng, liên kết trực tiếp với USMH Monitoring.
- CloudWatch là một kỹ năng/hệ thống liên kết với USMH Monitoring.
- SPPL Microservices platform là một kỹ năng/hệ thống liên kết với Chu Ngoc M.
- Azure là một kỹ năng/hệ thống liên kết với Chu Ngoc M.
- AWS là một kỹ năng/hệ thống liên kết với Chu Ngoc M.
- Linux là một kỹ năng/hệ thống liên kết với 